# Decorators

A decorator is a function that takes another function and extends the behavior of the latter function without explicitly modifying it.

In [1]:
def outer_function():
    message = "hello"

    def inner_function():
        print(message)
    return inner_function()

In [2]:
outer_function()

hello


## Closure

A closure is a function object that remembers values in the enclosing scope even if they are not present in memory. It "closes over" the variables from its enclosing lexical scope, allowing the function to access and manipulate those variables even after the scope where it was defined has finished executing.

In [3]:
def outer_function(x):
    def inner_function(y):
        return x + y
    return inner_function # notice that () are not used in this case

closure_instance = outer_function(10)
result = closure_instance(5)
print(result)

15


## Decorators

In [7]:
def decorator_func(original_func):
    def wrapper_func():
        return original_func()
    return wrapper_func

def display():
    print('display function ran')

decorated_display = decorator_func(display)
decorated_display()

display function ran


Decorating our functions allow us to add additional functionalities.

In [9]:
def decorator_func(original_func):
    def wrapper_func():
        print('wrapper executed this before'.format(original_func.__name__))
        return original_func()
    return wrapper_func

def display():
    print('display function ran')

decorated_display = decorator_func(display)
decorated_display()

wrapper executed this before
display function ran


We can also use @decorator_func

In [10]:
def decorator_func(original_func):
    def wrapper_func():
        print('wrapper executed this before'.format(original_func.__name__))
        return original_func()
    return wrapper_func

@decorator_func
def display():
    print('display function ran')

display()

wrapper executed this before
display function ran


In [12]:
def decorator_func(original_func):
    def wrapper_func():
        print('wrapper executed this before'.format(original_func.__name__))
        return original_func()
    return wrapper_func

@decorator_func
def display():
    print('display function ran')

@decorator_func
def display_info(name,age):
    print('display info ran with arguments ({},{})'.format(name,age))

display()
display_info('John',25)

wrapper executed this before
display function ran


TypeError: decorator_func.<locals>.wrapper_func() takes 0 positional arguments but 2 were given

Add *args, **kwargs to function

In [13]:
def decorator_func(original_func):
    def wrapper_func(*args, **kwargs):
        print('wrapper executed this before {}'.format(original_func.__name__))
        return original_func(*args, **kwargs)
    return wrapper_func

@decorator_func
def display():
    print('display function ran')

@decorator_func
def display_info(name,age):
    print('display info ran with arguments ({},{})'.format(name,age))

display()
display_info('John',25)

wrapper executed this before
display function ran
wrapper executed this before
display info ran with arguments (John,25)


## Classes as decorators

We can also use classes as decorators

In [16]:
def decorator_func(original_func):
    def wrapper_func(*args, **kwargs):
        print('wrapper executed this before'.format(original_func.__name__))
        return original_func(*args, **kwargs)
    return wrapper_func

class decorator_class(object):
    def __init__(self, original_func):
        self.original_func = original_func

    def __call__(self, *args, **kwargs):
        print('call method executed this before {}'.format(self.original_func.__name__))
        return self.original_func(*args, **kwargs)

@decorator_class
def display():
    print('display function ran')

@decorator_class
def display_info(name,age):
    print('display info ran with arguments ({},{})'.format(name,age))

display()
display_info('John',25)



call method executed this before display
display function ran
call method executed this before display_info
display info ran with arguments (John,25)


We'll stick to decorator functions

## Logging

Avoid having to implement logging functionality everytime you implement a function

In [21]:
def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    def wrapper(*args, **kwargs):
        logging.info('Ran wiith args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)
    return wrapper

# @decorator_func
# def display():
#     print('display function ran')

@my_logger
def display_info(name,age):
    print('display info ran with arguments ({},{})'.format(name,age))

# display()
display_info('John',25)

display info ran with arguments (John,25)


## Timer

In [23]:
def my_timer(orig_func):
    import time
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}sec".format(orig_func.__name__, t2))
        return result
    return wrapper

# @decorator_func
# def display():
#     print('display function ran')

import time

@my_timer
def display_info(name,age):
    time.sleep(1)
    print('display info ran with arguments ({},{})'.format(name,age))

# display()
display_info('John',25)

display info ran with arguments (John,25)
display_info ran in: 1.0020818710327148sec


In [24]:
import time

@my_timer
@my_logger
def display_info(name,age):
    time.sleep(1)
    print('display info ran with arguments ({},{})'.format(name,age))

# display()
display_info('John',25)

display info ran with arguments (John,25)
wrapper ran in: 1.0041089057922363sec


wrapper ran? we want display_info() ran in ...

In [25]:
@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print('display info ran with arguments ({},{})'.format(name,age))

# display()
display_info('John',25)

display info ran with arguments (John,25)
display_info ran in: 1.0022780895233154sec


In [26]:
from functools import wraps

def my_timer(orig_func):
    import time

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = orig_func(*args, **kwargs)
        t2 = time.time() - t1
        print("{} ran in: {}sec".format(orig_func.__name__, t2))
        return result
    return wrapper

def my_logger(orig_func):
    import logging
    logging.basicConfig(filename='{}.log'.format(orig_func.__name__), level=logging.INFO)

    @wraps(orig_func)
    def wrapper(*args, **kwargs):
        logging.info('Ran wiith args: {}, and kwargs: {}'.format(args, kwargs))
        return orig_func(*args, **kwargs)
    return wrapper

@my_logger
@my_timer
def display_info(name,age):
    time.sleep(1)
    print('display info ran with arguments ({},{})'.format(name,age))

# display()
display_info('Hank',25)

display info ran with arguments (Hank,25)
display_info ran in: 1.0036816596984863sec


In [27]:
print(display_info.__name__)

display_info
